In [ ]:
import time, sys
sys.path.append('..')
from pb_diff_envs.environment.static.rand_kuka_env import RandKukaEnv

import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import pybullet as p
import pb_diff_envs.objects.static.voxel_group as vv; reload(vv)
from pb_diff_envs.environment.static.kuka_rand_vgrp import Kuka_VoxelRandGroupList


## avg one traj len: , horizon = 
num_groups = 5
num_voxels = 5
hr = 0.20 # 0.2
vr_x = 0.2; vr_y = 0.2;vr_z = (0., 0.)
se_x = 0.5; se_y = 0.5; se_z = (0.3, 0.9) # 0.4 too difficult
hExt_range = [hr, hr, hr] # fixed-size obstacle
seed = 333

## check z
start_end = np.array([[-se_x, se_x], [-se_y, se_y], se_z])
hExt_range = np.array(hExt_range)
void_range = np.array( [(-vr_x, vr_x), (-vr_y, vr_y), vr_z] ) # (0., 0.)

trivial = dict(gen_data=True, 
               samples_per_env=200, gen_num_parts=1, 
               dataset_url='./dataset/luotest-k7d-ipynb.hdf5',
               eval_num_groups=2, eval_num_probs=2, ## dummy placeholder
               )
mazelist_config = dict(planner_timeout=30, 
                        interp_density=3,
                       min_episode_distance=4*np.pi,)
## Manually create a group of envs
dvg_group = Kuka_VoxelRandGroupList(RandKukaEnv, num_groups, num_voxels, start_end, void_range, orn_range=None, \
 hExt_range=hExt_range, is_static=True, seed=seed, mazelist_config=mazelist_config, **trivial)

## Create a Predefined Group of envs
# import gym
# dvg_group = gym.make('kuka7d-testOnly-v8', load_mazeEnv=False, gen_data=True) # False) # True



## simple vis of env

In [ ]:
if p.isConnected():
    p.disconnect()
env = dvg_group.create_single_env(3) # 
env.unload_env()
env.load(GUI=False) # if in a headless server: False; local machine: True
p.getContactPoints()
p.stepSimulation()
# env.robot.get_workspace_observation()
plt.imshow(env.render())
plt.show()
p.configureDebugVisualizer(p.COV_ENABLE_GUI, 0)  # Disable the default GUI controls
p.configureDebugVisualizer(p.COV_ENABLE_MOUSE_PICKING, 1)  # Enable mouse picking for camera control
p.addUserDebugLine([0, 0, 0], [1, 0, 0], lineColorRGB=[1, 0, 0], lineWidth=6)
p.addUserDebugLine([0, 0, 0], [0, 1, 0], lineColorRGB=[0, 1, 0], lineWidth=6)
p.addUserDebugLine([0, 0, 0], [0, 0, 1], lineColorRGB=[0, 0, 1], lineWidth=6)

no_while = True

while True:
    p.stepSimulation()
    # p.performCollisionDetection()
    c_pts = p.getContactPoints()
    # print(p.get)
    time.sleep(0.1)
    if c_pts is not None and len(c_pts) > 0:
        # print(p.getContactPoints())
        pass
        # time.sleep(2)
        # p.resetJointStatesMultiDof(env.robot_id, np.arange(7), [[0.]]*7)
    if no_while:
        break

In [ ]:
## vis distribution
from utils.save_utils import save_scatter_fig
# save_scatter_fig (env.wall_locations)
points = []
for _ in range(2000):
    pt, _ = dvg_group.cuboid_grp.sample_xyz_hExt()
    points.append(pt[:2])
save_scatter_fig(points, None, n_kuka=1)

In [ ]:
# env.unload_env()

In [ ]:
'''Only for debug, used with a connected monitor'''
if False:
    print(env)
    # p.getOverlappingObjects((0.01, .01, .01))
    p.performCollisionDetection()
    c_points = p.getContactPoints()
    for cp in c_points:
        print(cp)
    print(p.getNumJoints(2))
    print(p.getBasePositionAndOrientation(0))
    # p.disconnect()
    print(p.getConnectionInfo())
    print(p.getAABB(0)) # a bounding box, min point and max point
    print(p.getBodyInfo(2))

    plt.imshow(env.render())
    start_time = time.time()

    while True:
        time.sleep(0.05)
        elapsed_time = time.time() - start_time
        if elapsed_time > 10:
            start_time = time.time()
            env.robot.print_joint_states()
            print('no_collision:', env.robot.no_collision())
        p.stepSimulation()
        break
    print(f'env.robot.limits_high {env.robot.limits_high}')
    env.robot.print_joint_states()
    print(env.robot.item_id)
    print(env.robot_id) # list
    env.robot.limits_high
    # env.robot.limits_low
    print(env._get_joint_pose())
    print(f'{type(env._get_joint_pose())}')

## Sample a problem/episode

In [ ]:
from utils.utils import DotDict
from tqdm import tqdm
env.load(GUI=False) # if in a headless server: False; local machine: True
prev_pose = np.array([0.0] * 7)
env.robot.set_config(prev_pose)
solutions = []
n_traj = 2 # 20
for i in tqdm(range(n_traj)):
    ## this solution is a sparse trajectory
    solution, _ = env.sample_1_episode(prev_pose)
    solution = np.array(solution)
    prev_pose = solution[-1]
    solutions.append(solution)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pybullet as p
import pybullet_data
import numpy as np
from utils.utils import save_gif
from IPython.display import HTML
import base64
from objects.trajectory import WaypointLinearTrajectory, WaypointProportionTrajectory
from utils.kuka_utils_luo import SolutionInterp
from importlib import reload
import utils.robogroup_utils_luo  as rul; reload(rul)
from utils.robogroup_utils_luo import robogroup_visualize_traj_luo
from utils.kuka_utils_luo import visualize_kuka_traj_luo

noise_config = {'std': 0.000}

sol_interp = SolutionInterp(density=mazelist_config['interp_density'])
sol_i = solutions[0] # which motion trajectory to visualize
result = DotDict(solution=sol_i)
traj_vis = sol_interp(result.solution,)

gifs, ds, vis_dict = visualize_kuka_traj_luo(env, traj_vis, is_debug=False)

print(traj_vis.shape)
print(result.solution.shape)

gif_name = f'./test_bit_kuka.gif'
save_gif(gifs, gif_name, duration=ds)
b64 = base64.b64encode(open(gif_name, 'rb').read()).decode('ascii')
display(HTML(f'<img src="data:image/gif;base64,{b64}" />'))

# import mediapy as mpy
# gifs = np.array(gifs)
# gifs.shape
# mpy.show_video(gifs[:, :,:,:3], fps=10)
